In [2]:
import time, enum, math
import numpy as np
import pandas as pd
import pylab as plt
from mesa import Agent, Model
from mesa.time import SimultaneousActivation, RandomActivation
from mesa.space import NetworkGrid
from mesa.datacollection import DataCollector
from networkx.algorithms.shortest_paths.generic import has_path
import networkx as nx
import panel as pn
import panel.widgets as pnw
import random

In [3]:
def kmtoNaut(km):
    return km / 1.852

In [93]:
ports = pd.read_csv('../ports.csv')
data = pd.read_csv('../distances.csv')
origin = pd.read_csv('../origin_dest.csv', sep=";")

In [94]:
ports.head()

,Unnamed: 0,PORT_NAME,INDEX_NO,coords
0,49159,Terminal Pesquero Cta. Quiane,NaN,"((-70.31722387298942, -18.513597026467323),)"
1,49164,Oil Berth,NaN,"((-61.86886473007713, 17.150384410999997),)"
2,16,Port of Basamuk,NaN,"((146.14295817405977, -5.53913255687803),)"
3,26,Victoria,NaN,"((-123.32715191091728, 48.402783083729446),)"
4,34,NaN,NaN,"((126.50786074843957, 36.333661512471735),)"


In [92]:
origin.head()

,table,Port_Name,Ref,Port_Ref,Country,1000TEU (2017),PROB (TAU_port / TAU_all) 2017
0,1,Shanghai,NaN,Shanghai Gang,China,40233,"0,085759048"
1,2,Singapore,6859.0,NaN,Singapore,33666,"0,071761095"
2,3,Shenzhen,NaN,Shenzhen,Gang,25208,"0,053732361"
3,4,Ningbo-Zhoushan,NaN,ZHOUSHAN,China,24607,"0,052451294"
4,5,Hong Kong,NaN,HONG KONG,China,20770,"0,044272499"


In [95]:
origin.merge(ports, left_on='Port_Ref', right_on='Unnamed: 0')

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [9]:
N = data["next_port"].tolist()
print(len(N))
N = list(set(N))
print(len(N))

56154
2842


In [5]:
# distances = [
#     ['CAP', 'ROT', 11367.07658],
#     ['CAP', 'SUE', 9814.514498],
#     ['CAP', 'SHA', 14090.03741],
#     ['ROT', 'PSA', 6013.579478],
#     ['PSA', 'SUE', 162.274111],
#     ['SHA', 'SUE', 13162.53259]]

In [11]:
distances = data[["prev_port", "next_port", "distance"]]

In [21]:
distances.astype({'prev_port':'int64', 'next_port':'int64'}).dtypes

prev_port      int64
next_port      int64
distance     float64
dtype: object

In [43]:
G = nx.MultiGraph()
G.add_nodes_from(N) #instatiate the ports as nodes of network
for i in range(len(distances)): #create bi-directional edges with an attribute for length 
       G.add_edge(distances.iloc[i][0], distances.iloc[i][1], length=distances.iloc[i][2])
grid = NetworkGrid(G) #Define Mesa Grid as the just created Network to allow for shipping only in routes

In [6]:
# for i in range(len(distances)):
#     distances[i][2] = kmtoNaut(distances[i][2])

In [64]:
class ShippingNetwork(Model):
    def __init__(self, N, distances, S=2):
        self.ports = N
        self.num_ships = S
        self.distances = distances
        self.schedule = SimultaneousActivation(self)
        self.running = True

        self.G = nx.MultiGraph()
        self.G.add_nodes_from(self.ports) #instatiate the ports as nodes of network
        for i in range(len(distances)): #create bi-directional edges with an attribute for length 
            self.G.add_edge(distances.iloc[i][0], distances.iloc[i][1], length=distances.iloc[i][2])
        self.grid = NetworkGrid(self.G) #Define Mesa Grid as the just created Network to allow for shipping only in routes

        #create ability to remove edges mid-model
        def network_change(self, change_type, change_edge):
            if change_type == "add":
                self.G.add_edge(self.change_edge[0], self.change_edge[1], length=self.change_edge[2])
            if change_type == "remove":
                self.G.remove_edge(self.change_edge[0],self.change_edge[1]) #can we identify an edge by node 1, node 2 & distance?
            #update model with new grid    
            return NetworkGrid(self.G), G


        #create agents 
        Ships = []
        for i in range(self.num_ships):
            port = random.sample(self.ports, k=2) # this is a test - implementation, more complex algorithming is necessary
            a = Ship(i+1, self, self.G, port[0], port[1])
            self.schedule.add(a)
            #append to list of ships
            Ships.append(a)
            #place agent on origin node
            self.grid.place_agent(a, port[0])


        self.datacollector = DataCollector(
            model_reporters={"Ports": "ports", "Graph":"G"},
            agent_reporters={"Position": "position", "Destination":"destination", "Itinerary":"itinerary", "Distance_Traveled":"distance_traveled", "route":"current_route" })


    def step(self, change_type='', change_edge=[]):
        self.datacollector.collect(self)
        #check network for changes
        if change_type != '' :
            self.grid, self.G = network_change(change_type, change_edge)

        self.schedule.step()     #Run each Agents


In [124]:
class Ship(Agent):
    def __init__(self, unique_id, model, G, start_port, destin_port, s=13.0):
        super().__init__(unique_id, model)
        self.destination = destin_port
        self.state = 0 #0 for active, numbers > 0 for weeks that ships have to "wait" until arrival to port
        self.speed = s*24*7 #speed is given in knots, with 1 knot being 1 nautical mile per hour
        self.position = start_port
        self.init_route = nx.dijkstra_path(G, self.position, self.destination, weight='length') #We keep a copy of the entire itinerary / distance traveled
        self.init_dist = nx.dijkstra_path_length(G,self.position, self.destination, weight='length') 
        self.current_route, self.current_dist = self.init_route, self.init_dist  #For comparison & navigational purposes, we use current route & distance
        self.next_position = self.current_route[1]
        self.itinerary = [self.position]
        self.distance_traveled = 0
        self.unique_id = unique_id
        self.G = G
        self.step_size = self.ident_distance()
        self.route_chng = 0
    

    def routing(self):
        #implement dijkstra to define shortest route
        route = nx.dijkstra_path(self.G, self.position, self.destination, weight='length')
        travel_distance = nx.dijkstra_path_length(self.G,self.position, self.destination, weight='length')
        #include a way for capacity?
        return route, travel_distance

    def move(self):
        self.next_position = self.current_route[1]
        self.step_size = self.ident_distance() #look up the distance between two cities 
        self.state = self.step_size / self.speed #change state to step amount
        self.current_dist = self.current_dist - self.step_size #adjust current distance minus the distance traveled in the next step
        self.model.grid.move_agent(self, self.next_position) #move the agent
        self.current_route.remove(self.current_route[0]) #remove the next step from the itinerary
        self.position = self.next_position
        if len(self.current_route) == 1:
            self.next_position = self.current_route[0] 
        else:
            self.next_position = self.current_route[1] #update current route

    def ident_distance(self): #look up the distance of the current step
        return self.G.get_edge_data(self.position, self.next_position)[0]['length']
    
    def reroute(self):
         pass

    def step(self):
        self.state = self.state - 1 #'move' ships by one weeks progress
        if self.state <= 0.000: #ships that are en-route to the node they are going to next do not move / perform other activities
            self.distance_traveled += self.step_size #ship has arrived at port, let's add the distance traveled to their 
           
            #add the current position to itinerary
            if self.position != self.destination: #if current stop is not the final stop
                new_route, new_distance = self.routing() #perform a new routing to compare against current routing
                
                if new_route == self.current_route: #if current routing is the same as new, just move (default case)
                    print("default case")
                    self.move()
                    self.itinerary.append(self.position)
        
            # THIS CURRENTLY ONLY CHANGES THE ROUTE IF THE NEXT STEP IS BLOCKED
                elif new_distance > self.current_dist: #if current route is shorter than newly calculated route, check for obstructions
                    print('reroute: current route shorter than new route')
                    if not has_path(self.G, self.position, self.next_position): 
                        self.current_route = new_route
                        self.current_dist = new_distance
                        self.route_chng += 1
                    self.move()
                    self.itinerary.append(self.position)
                
                
                else: # final option is that current route is longer than new route (think Suez reopening after a while), here, we just take the new option
                    print('reroute: current route longer than new route')
                    self.current_route = new_route
                    self.current_dist = new_distance
                    self.move()
                    self.itinerary.append(self.position)
            
            else: #if ship is arrived at final position, get a new route, and start back
                print('arrival')
                self.destination = self.reroute()
                self.move()
                self.itinerary.append(self.position)
        print("Ship: {}, Source: {}, Destination: {}, Position: {},  Next Stop {}, Time until next Stop {}".format(self.unique_id, self.itinerary[0], self.destination, self.position, self.next_position, self.state ))

    
    # def collect_time

    # def collect_costs

In [125]:

model = ShippingNetwork(N, distances, 1)
steps = 20
for i in range(steps):
    print("\n", "Step: ", i)
    model.step()
agent_state = model.datacollector.get_agent_vars_dataframe()
agent_state


 Step:  0
default case
Ship: 1, Source: 2939, Destination: 3688, Position: 2040.0,  Next Stop 6868.0, Time until next Stop 0.17137985834056998

 Step:  1
default case
Ship: 1, Source: 2939, Destination: 3688, Position: 6868.0,  Next Stop 6869.0, Time until next Stop 2.0617668269297322

 Step:  2
Ship: 1, Source: 2939, Destination: 3688, Position: 6868.0,  Next Stop 6869.0, Time until next Stop 1.0617668269297322

 Step:  3
Ship: 1, Source: 2939, Destination: 3688, Position: 6868.0,  Next Stop 6869.0, Time until next Stop 0.06176682692973223

 Step:  4
default case
Ship: 1, Source: 2939, Destination: 3688, Position: 6869.0,  Next Stop 7100.0, Time until next Stop 0.0005984462812785849

 Step:  5
default case
Ship: 1, Source: 2939, Destination: 3688, Position: 7100.0,  Next Stop 27973.0, Time until next Stop 1.577854681949348

 Step:  6
Ship: 1, Source: 2939, Destination: 3688, Position: 7100.0,  Next Stop 27973.0, Time until next Stop 0.577854681949348

 Step:  7
default case
Ship: 1, 

IndexError: list index out of range

In [126]:
agent_state = model.datacollector.get_agent_vars_dataframe()
agent_state


,,Position,Destination,Itinerary,Distance_Traveled,route
Step,AgentID,,,,,
0,1,2939.0,3688,"[2939, 2040.0, 6868.0, 6869.0, 7100.0, 27973.0...",0.000000,[3688.0]
1,1,2040.0,3688,"[2939, 2040.0, 6868.0, 6869.0, 7100.0, 27973.0...",374.293611,[3688.0]
2,1,6868.0,3688,"[2939, 2040.0, 6868.0, 6869.0, 7100.0, 27973.0...",748.587221,[3688.0]
3,1,6868.0,3688,"[2939, 2040.0, 6868.0, 6869.0, 7100.0, 27973.0...",748.587221,[3688.0]
4,1,6868.0,3688,"[2939, 2040.0, 6868.0, 6869.0, 7100.0, 27973.0...",748.587221,[3688.0]
5,1,6869.0,3688,"[2939, 2040.0, 6868.0, 6869.0, 7100.0, 27973.0...",5251.485971,[3688.0]
6,1,7100.0,3688,"[2939, 2040.0, 6868.0, 6869.0, 7100.0, 27973.0...",5252.792978,[3688.0]
7,1,7100.0,3688,"[2939, 2040.0, 6868.0, 6869.0, 7100.0, 27973.0...",5252.792978,[3688.0]
8,1,27973.0,3688,"[2939, 2040.0, 6868.0, 6869.0, 7100.0, 27973.0...",8698.827603,[3688.0]


In [127]:
route = nx.dijkstra_path(G,2939, 3688, weight='length')
route

[2939, 2040.0, 6868.0, 6869.0, 7100.0, 27973.0, 2876.0, 3268.0, 3688.0]

In [123]:
 travel_distance = nx.dijkstra_path_length(G,5561, 6165, weight='length')
 travel_distance *

597.5009531952306

In [88]:
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
cmap = ListedColormap(["lightblue", "orange", "green",])

def plot_grid(model,fig,layout='spring',title=''):
    graph = model.G
    if layout == 'kamada-kawai':      
        pos = nx.kamada_kawai_layout(graph)  
    elif layout == 'circular':
        pos = nx.circular_layout(graph)
    else:
        pos = nx.spring_layout(graph, iterations=5, seed=8)  
    plt.clf()
    ax=fig.add_subplot()

    nx.draw(graph, pos, node_size=100, edge_color='gray', with_labels=True,
            alpha=0.9,font_size=14,ax=ax)
    ax.set_title(title)
    return

#example usage
fig,ax=plt.subplots(1,2,figsize=(16,10))
model = ShippingNetwork(N, distances)
model.step(distances)
f=plot_grid(model,fig,layout='kamada-kawai')

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [24]:
model.schedule.

[]